Data importation:
We import the packages needed and the datasets to use

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
categories_df=pd.read_json("https://raw.githubusercontent.com/anyoneai/e-commerce-open-data-set/master/categories.json")
products=pd.read_json("https://raw.githubusercontent.com/anyoneai/e-commerce-open-data-set/master/products.json")


In [2]:
categories_df.head(5)

,id,name,path,subCategories
0,abcat0010000,Gift Ideas,"[{'id': 'abcat0010000', 'name': 'Gift Ideas'}]","[{'id': 'pcmcat140000050035', 'name': 'Capturi..."
1,abcat0020001,Learning Toys,"[{'id': 'abcat0010000', 'name': 'Gift Ideas'},...",[]
2,abcat0020002,DVD Games,"[{'id': 'abcat0010000', 'name': 'Gift Ideas'},...",[]
3,abcat0020004,Unique Gifts,"[{'id': 'abcat0010000', 'name': 'Gift Ideas'},...",[]
4,abcat0100000,TV & Home Theater,"[{'id': 'abcat0100000', 'name': 'TV & Home The...","[{'id': 'abcat0101000', 'name': 'TVs'}, {'id':..."


In [3]:
products.head(5)

,sku,name,type,price,upc,category,shipping,description,manufacturer,model,url,image
0,43900,Duracell - AAA Batteries (4-Pack),HardGood,5.49,41333424019,"[{'id': 'pcmcat312300050015', 'name': 'Connect...",5.49,Compatible with select electronic devices; AAA...,Duracell,MN2400B4Z,http://www.bestbuy.com/site/duracell-aaa-batte...,http://img.bbystatic.com/BestBuy_US/images/pro...
1,48530,Duracell - AA 1.5V CopperTop Batteries (4-Pack),HardGood,5.49,41333415017,"[{'id': 'pcmcat312300050015', 'name': 'Connect...",5.49,Long-lasting energy; DURALOCK Power Preserve t...,Duracell,MN1500B4Z,http://www.bestbuy.com/site/duracell-aa-1-5v-c...,http://img.bbystatic.com/BestBuy_US/images/pro...
2,127687,Duracell - AA Batteries (8-Pack),HardGood,7.49,41333825014,"[{'id': 'pcmcat312300050015', 'name': 'Connect...",5.49,Compatible with select electronic devices; AA ...,Duracell,MN1500B8Z,http://www.bestbuy.com/site/duracell-aa-batter...,http://img.bbystatic.com/BestBuy_US/images/pro...
3,150115,Energizer - MAX Batteries AA (4-Pack),HardGood,4.99,39800011329,"[{'id': 'pcmcat312300050015', 'name': 'Connect...",5.49,4-pack AA alkaline batteries; battery tester i...,Energizer,E91BP-4,http://www.bestbuy.com/site/energizer-max-batt...,http://img.bbystatic.com/BestBuy_US/images/pro...
4,185230,Duracell - C Batteries (4-Pack),HardGood,8.99,41333440019,"[{'id': 'pcmcat312300050015', 'name': 'Connect...",5.49,Compatible with select electronic devices; C s...,Duracell,MN1400R4Z,http://www.bestbuy.com/site/duracell-c-batteri...,http://img.bbystatic.com/BestBuy_US/images/pro...


We will set the features to be the X from now and the labels to be the y

In [4]:
import numpy as np
from sklearn.preprocessing import OrdinalEncoder

new_category_list=[]
for x in products.category:
    new_category_list.append(list(x[-1].values())[1])

products.category=new_category_list
y = products.category
enc = OrdinalEncoder(dtype='int')
y = np.array(y)
enc.fit(y.reshape(-1,1))
aux = enc.transform(y.reshape(-1,1))
aux=pd.DataFrame(aux)

new_categories=[]
threshold=32
values=aux.value_counts().values
indexes=aux.value_counts().index

for x,y in zip(indexes,values):
    if y > threshold:
        new_categories.append(x[0])
    else:
        new_categories.append('other')

new_y=[]
for category in aux.values:
    if category[0] in new_categories:
        new_y.append(category[0])
    else:
        new_y.append('other')

X = products.drop(['category'], axis=1)
y = new_y
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.3, random_state=42)

In [5]:
categories_list=[]
for category in new_y:
    if type(category)!=str:
        label=enc.categories_[0][category]
        if label not in categories_list:
            categories_list.append(label)
import numpy as np
np.set_printoptions(threshold=sys.maxsize)
np.array(categories_list)

array(['Alkaline Batteries', 'Specialty Batteries',
       'Dash Installation Kits', 'Deck Harnesses', 'Antennas & Adapters',
       'Cordless Phone Batteries', 'Car Subwoofers',
       'Sound Recording Software', 'Music Memorabilia',
       'Keyboard Accessories', 'More Pop Culture Merchandise',
       'Audio Interfaces', 'Microphones', 'DJ Equipment Accessories',
       'Recording & Music Stands', 'Sheet Music', 'Wii Games',
       'DSLR Body & Lens', 'Xbox 360 Games', 'Action Camcorder Mounts',
       'DJ Lighting & Effects', 'Battery Backup (UPS)', 'Pre-Owned Games',
       'Powersports Audio', 'Monitor & Video Cables', 'Cooktops',
       'All Headphones', 'Coffee Pods', 'Single-Serve Blenders',
       'Upright Freezers', 'Projector Screens', 'Power Supplies',
       'Hair Dryers', 'Over-Ear Headphones', 'Accessories',
       'Nintendo DS Games', 'PCI Express Graphics Cards', 'PS3 Games',
       'TV Mounts', 'Condenser', 'Prime Lenses',
       'Laptop Chargers & Adapters', 'Window 

In [6]:
import pickle
pickle.dump(enc, open(b"encoder.pkl","wb"))


Now we create some classes to be used as esential parts in a future pipeline in the file Utils.py

In [7]:
import nltk
nltk.download("punkt")
nltk.download("wordnet")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\frodr\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\frodr\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\frodr\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### We create the definitive pipeline for preprocessing

In [8]:

import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, MinMaxScaler, OrdinalEncoder
from utils import Word2VecProcessor, FastTextProcessor, DropColumns, ColumnExtractor, MyOneHotEncoding, NameDescriptionImputation, NameDescriptionNormalization, TfidfVectorizerTransformer, OrdinalEncoding
pd.options.mode.chained_assignment = None
import pickle

# Transformers
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, MinMaxScaler

# Modeling Evaluation
from sklearn.model_selection import train_test_split, cross_val_score, KFold, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score,f1_score, confusion_matrix, classification_report
from IPython.display import display, Markdown

# Pipelines
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

#liraries for NLP
import os
import sys
# I had some issues importing files
# This is how I addressed them:
cwd = os.getcwd()
add = "/".join(cwd.split("/")[:-1])
sys.path.append(add)

from text_normalizer import normalize_corpus
from sklearn.feature_extraction.text import TfidfVectorizer


COLS_TO_DROP = ['sku','upc', 'shipping','manufacturer', 'model', 'url', 'image']
pipe = Pipeline([
    ('drop columns', DropColumns(COLS_TO_DROP)),
    ('features', FeatureUnion([
        ('price', Pipeline([
            ('extract', ColumnExtractor(["price"])),
            ('imputation', SimpleImputer(strategy="mean")),
            ('scaler', StandardScaler())
        ])),
        ('type', Pipeline([
            ('extract', ColumnExtractor(["type"])),
            ('imputation', SimpleImputer(strategy="most_frequent")),
            ('ohe', OneHotEncoder(drop='first'))
        ])),
        ('name and description', Pipeline([
            ('extract', ColumnExtractor(["name","description"])),#"description"])),
            ('imputation', NameDescriptionImputation()),
            ('text normalizer', NameDescriptionNormalization()),    
            ('text vectorizer', TfidfVectorizerTransformer())
        ]))
    ]))
])


pipe_name = Pipeline([
        ('name and description', Pipeline([
            ('extract', ColumnExtractor(["name","description"])),#"description"])),
            ('imputation', NameDescriptionImputation()),
            ('text normalizer', NameDescriptionNormalization()),    
            #('text word2vec', Word2VecProcessor()) #('text vectorizer', TfidfVectorizerTransformer())
            ('text vectorizer', TfidfVectorizerTransformer())
        ]))
])

pipeline_for_predict=Pipeline([
        ('name and description', Pipeline([
            ('imputation', NameDescriptionImputation()),
            ('text normalizer', NameDescriptionNormalization()),    
            ('text vectorizer', TfidfVectorizerTransformer())
        ]))
])

### We test it

In [9]:
pipe_name.fit(X_train, y_train)

X_train1=pipe_name.transform(X_train)
X_test1=pipe_name.transform(X_test)

#pipeline_for_predict.fit(X_train, y_train)

In [10]:
a={'name':'AA Bateries','description':'Bateries for all uses'}
test = pd.DataFrame([a])
test
pipe_name.transform(test)

<1x14827 sparse matrix of type '<class 'numpy.float64'>'
	with 1 stored elements in Compressed Sparse Row format>

### Now we export the pipeline using Pickle library

In [11]:
import pickle
filehandler = open(b"pipeline_name.pkl","wb")
pickle.dump(pipe_name,filehandler)

In [12]:
pipe.fit_transform(X_train, y_train)

X_train=pipe.transform(X_train)
X_test=pipe.transform(X_test)

### Now we try different models to get a first Baseline Model

#### Random Forest

In [13]:

from sklearn.ensemble import RandomForestClassifier

labels=list(enc.get_params())

cl_rf = RandomForestClassifier(n_jobs=-1)
cl_rf.fit(X_train1, y_train)
y_preds = cl_rf.predict(X_test1)
y_probas=cl_rf.predict_proba(X_test1)
f1_score(y_test, y_preds, average="weighted")
micro=f1_score(y_test, y_preds, average=None)


In [14]:
f1_score(y_test, y_preds, average="weighted")

0.8334349873405688

#### K Neighbors

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
KNN_model = KNeighborsClassifier(n_neighbors=3).fit(X_train1, y_train)
KNN_pred = KNN_model.predict(X_test1)  
print(f1_score(y_test, KNN_pred, average="weighted"))

0.7372579233423867


In [ ]:
proba=KNN_model.predict_proba(X_test1)

In [ ]:
pickle.dump(KNN_model,open(b"KNN_model.pkl","wb"))

array(['other', '881', '101', ..., '400', '1427', '997'], dtype='<U11')

#### Linear SVM

In [16]:
from sklearn.svm import LinearSVC
svm_model = LinearSVC(max_iter=3000).fit(X_train1, y_train)
svm_pred = svm_model.predict(X_test1)  
f1_score(y_test, svm_pred, average="weighted")

c:\Users\frodr\anaconda3\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


0.7891845551398722